<a href="https://colab.research.google.com/github/FinancialEngineerLab/finefinance/blob/master/Volatility_Forecasting_with_Univariate_Vol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



---

Volatility Forecasting 



---



---



---


Author : Hyunjin Shin

# Analytical
# Simulaton
# Bootstrap

In [ ]:
!pip install arch
!pip install Quandl
!pip install -U finance_datareader

Requirement already up-to-date: finance_datareader in /usr/local/lib/python3.6/dist-packages (0.9.6)


In [ ]:
### Load Library ###

In [ ]:
import numpy as np
import pandas as pd
import sys
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import warnings
import seaborn as sns
import datetime as dt
import scipy.stats as ss
import quandl
import io
from arch import arch_model
from pylab import *

In [ ]:
### Data Preprocessing ###

In [ ]:
### Data ###

now = datetime.datetime.today().strftime('%Y-%m-%d')

def dataframe(ticker, start_date = '2008-01-01',end_date= now):
    # Series Generation
    df_list_1 = fdr.DataReader(ticker,  "2008-01-01","2008-12-31")[['Open','High','Low','Close']]
    df_list_2 = fdr.DataReader(ticker,  "2009-01-01","2009-12-31")[['Open','High','Low','Close']]
    df_list_3 = fdr.DataReader(ticker,  "2010-01-01","2010-12-31")[['Open','High','Low','Close']]
    df_list_4 = fdr.DataReader(ticker,  "2011-01-01","2011-12-31")[['Open','High','Low','Close']]
    df_list_5 = fdr.DataReader(ticker,  "2012-01-01","2012-12-31")[['Open','High','Low','Close']]
    df_list_6 = fdr.DataReader(ticker,  "2013-01-01","2013-12-31")[['Open','High','Low','Close']]
    df_list_7 = fdr.DataReader(ticker,  "2014-01-01","2014-12-31")[['Open','High','Low','Close']]
    df_list_8 = fdr.DataReader(ticker, "2015-01-01","2015-12-31")[['Open','High','Low','Close']]
    df_list_9 =fdr.DataReader(ticker,  "2016-01-01","2016-12-31")[['Open','High','Low','Close']]
    df_list_10 =fdr.DataReader(ticker, "2017-01-01","2017-12-31")[['Open','High','Low','Close']]
    df_list_11 =fdr.DataReader(ticker, "2018-01-01","2018-12-31")[['Open','High','Low','Close']]
    df_list_12 =fdr.DataReader(ticker, "2019-01-01","2019-12-31")[['Open','High','Low','Close']]
    df_list_13 =fdr.DataReader(ticker, "2020-01-01")[['Open','High','Low','Close']]
                
    ### data frame preprocessing ###
    # axis transformation
    # axis = 1이면 cbind, axis = 0이면 rbind
    df_total = pd.concat([df_list_1, df_list_2, df_list_3, df_list_4,df_list_5, df_list_6, df_list_7, df_list_8,
                          df_list_9, df_list_10, df_list_11, df_list_12, df_list_13])
    df_total = df_total[(df_total.index >= start_date) & (df_total.index < end_date)]
    # NA cleaning
    df_raw0 = df_total[['Open','High','Low']]
    df_raw = df_total['Close']
    df_raw = df_raw.fillna(method = 'ffill')
    df_raw0 = df_raw0.fillna(method = 'ffill')
    df = pd.concat([df_raw0,df_raw], axis =1)

    df.columns = ["Open","High","Low","Close"]
    df['returns'] = df.Close.pct_change().dropna()

    return df

In [ ]:
data = dataframe('KS200', start_date = '2017-01-01')
data.describe()
data.tail(5)

,Open,High,Low,Close,returns
Date,,,,,
2020-05-29,266.62,269.61,265.23,268.32,0.000112
2020-06-01,269.29,273.39,268.93,273.19,0.018150
2020-06-02,272.65,276.47,272.22,276.08,0.010579
2020-06-03,279.14,287.49,278.98,285.91,0.035606
2020-06-04,291.17,292.90,284.93,286.45,0.001889




---



---

# Basic Forecasting #


---



---

ARCH Model

In [ ]:
returns = data['returns']
returns

Date
2017-01-02         NaN
2017-01-03    0.010025
2017-01-04    0.000570
2017-01-05   -0.004333
2017-01-06    0.004657
                ...   
2020-05-29    0.000112
2020-06-01    0.018150
2020-06-02    0.010579
2020-06-03    0.035606
2020-06-04    0.001889
Name: returns, Length: 838, dtype: float64

In [ ]:
returns = data['returns'].dropna()
price = data['Close']
# ARCH (1,0,1)
am = arch_model(price, vol='Garch', p=1, o=0, q=1, dist='Normal')
res = am.fit(update_freq=5)
forecasts = res.forecast(horizon=5)

print(price.iloc[-2:])
print(forecasts.mean.iloc[-2:])
print(forecasts.residual_variance.iloc[-2:])
print(forecasts.variance.iloc[-2:])

Iteration:      5,   Func. Count:     35,   Neg. LLF: 3492.543318327739
Iteration:     10,   Func. Count:     69,   Neg. LLF: 3469.8481793851824
Iteration:     15,   Func. Count:    100,   Neg. LLF: 3468.346317285207
Iteration:     20,   Func. Count:    130,   Neg. LLF: 3468.212900962939
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3468.21122114012
            Iterations: 22
            Function evaluations: 142
            Gradient evaluations: 22
Date
2020-06-03    285.91
2020-06-04    286.45
Name: Close, dtype: float64
                   h.1         h.2         h.3         h.4         h.5
Date                                                                  
2020-06-03         NaN         NaN         NaN         NaN         NaN
2020-06-04  277.322827  277.322827  277.322827  277.322827  277.322827
                  h.1        h.2        h.3         h.4         h.5
Date                                                               
2020-0